In [1]:
import numpy as np
import scipy
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import math
import random
import json
import pickle
import matplotlib.pyplot as plt

# 1.读数据-t3.txt  毕克预处理后的文件


* t3.txt 是洗完之后的数据，点离散化了，
一行两个数，点的范围是[0, 16310)，一共有44805533条边
* dict.txt 是一个映射关系，表示原始ID和当前标号之间的映射关系。
可以根据这里的ID在 https://tweeterid.com/ 得到name然后去看具体的twitter

* 具体洗数据的方法是
    * 计算平均度数（2*边数/点数）
    * 删掉所有度数小于平均度数的点，和这些点关联的边。
    * 重复上述过程。

我最后洗出的数据每一个点的度数都>=1427

为了方便后续程序使用，我将所有用户ID做了一个映射，映射到了[0, 16310)的范围。

In [8]:
edge_i = []
edge_j = []
N = 0# number of nodes
iter_n = 0
stop_n = 90000000
f_name = "/home/ubuntu/data/t3.txt"
f = open(f_name,"r")


for line in f.xreadlines():
    tmp = line.strip("\n").split(" ")
    edge_i.append(int(tmp[0]))
    edge_j.append(int(tmp[1]))
    N = max(N,int(tmp[0]))
    N = max(N,int(tmp[1]))
    iter_n += 1
    if (iter_n%10000000==0):
        print iter_n/1000000
#     if (iter_n==stop_n):
#         break
print "Done"
nodeij = zip(edge_i,edge_j)
# with open('nodeij', 'r') as handle:
#     nodeij = pickle.load(handle)

KeyboardInterrupt: 

In [2]:
f_n = "/home/ubuntu/data/dict.txt"
with open(f_n) as f:
    line = f.readlines()
line = line[0][1:(-2)]#remove { and }
items = line.split(",")
id_dict = {}
for item in items:
    item = item.replace(" ","").split(":")
    id_dict[int(item[1])]=int(item[0])
print "Length of the dict:",len(id_dict)# key is the transformed, and value is the original

Length of the dict: 16310


# 2. 建立稀疏矩阵并计算SVD

In [9]:
# data = [1.0] * len(edge_i)
# sparse_m = scipy.sparse.coo_matrix((data, (edge_i, edge_j)), shape=(N+1,N+1))

In [10]:
# singular_k = 10
# svd_result = svds(sparse_m, k=singular_k)
# u,s,v = svd_result[0],svd_result[1],svd_result[2]
# print u.shape
# print s.shape
# print v.shape

# with open('u_10_t3.pickle', 'wb') as handle:
#     pickle.dump(u, handle)

# with open('v_10_t3.pickle', 'wb') as handle:
#     pickle.dump(v, handle)
    
# with open('s_10_t3.pickle', 'wb') as handle:
#     pickle.dump(s, handle)

(16310, 10)
(10,)
(10, 16310)


In [3]:
# 读取SVD结果的数据
singular_k = 10
with open('v_10_t3.pickle', 'r') as handle:
    v = pickle.load(handle)
    
with open('u_10_t3.pickle', 'r') as handle:
    u = pickle.load(handle)

with open('s_10_t3.pickle', 'r') as handle:
    s = pickle.load(handle)
    
print u.shape
print s.shape
print v.shape
print s

(16310, 10)
(10,)
(10, 16310)
[  336.35946696   364.43392137   376.46544011   436.46074881   511.5340689
   555.32597352   747.91973802   855.85487872  2656.07712352  3468.32227425]


In [38]:
# 画出singular vector的图
plot_dir = "plot/"
for i in range(0,singular_k):
    for j in range(0,singular_k):
        if i>=j:
            continue
        fig = plt.figure()
        plt.plot(u[:,i],u[:,j],'bo',[-0.00002,0.00002],[0,0],'r--',[0,0],[-0.00002,0.00002],'r--')
        plt.xlabel(str(i))
        plt.ylabel(str(j))
        t = 'U_'+str(i)+"_"+str(j)
        plt.title(t)
        fig.savefig(plot_dir+t+".png")
        plt.close(fig)
    print "Plot U Iteration:",i

Plot Iteration: 0
Plot Iteration: 1
Plot Iteration: 2
Plot Iteration: 3
Plot Iteration: 4
Plot Iteration: 5
Plot Iteration: 6
Plot Iteration: 7
Plot Iteration: 8
Plot Iteration: 9


In [42]:
# 画出singular vector的图
plot_dir = "plot/"
for i in range(0,singular_k):
    for j in range(0,singular_k):
        if i>=j:
            continue
        fig = plt.figure()
        plt.plot(v[i,],v[j,],'bo',[-0.00002,0.00002],[0,0],'r--',[0,0],[-0.00002,0.00002],'r--')
        plt.xlabel(str(i))
        plt.ylabel(str(j))
        t = 'V_'+str(i)+"_"+str(j)
        plt.title(t)
        fig.savefig(plot_dir+t+".png")
        plt.close(fig)
    print "Plot V Iteration:",i

Plot V Iteration: 0
Plot V Iteration: 1
Plot V Iteration: 2
Plot V Iteration: 3
Plot V Iteration: 4
Plot V Iteration: 5
Plot V Iteration: 6
Plot V Iteration: 7
Plot V Iteration: 8
Plot V Iteration: 9


# 3. 选取初始seeds并找出问题节点

In [4]:
def select_seeds(u,ii,jj):
    krou,ktheta = 5,5
    vectorij = []
    for arr in u:
        vectorij.append([float(arr[ii-1]),float(arr[jj-1])])
    N = len(vectorij)
    routhetaij = []
    minrou,maxrou = 1.0,-1.0
    mintheta,maxtheta = -90.0,90.0
    for [entryi,entryj] in vectorij:
        rou = (entryi**2+entryj**2)**0.5
        if entryi == 0:
            if entryj > 0: theta = 90.0
            if entryj < 0: theta = -90.0
            if entryj == 0: theta = 0.0
        else:
            theta = math.atan(entryj/entryi)/math.pi*180.0
        routhetaij.append([rou,theta])
        minrou,maxrou = min(minrou,rou),max(maxrou,rou)
    gaprou,gaptheta = (maxrou-minrou)/krou,(maxtheta-mintheta)/ktheta
    freqrou,freqtheta = [0 for i in range(0,krou)],[0 for i in range(0,ktheta)]
    freq = [[0 for j in range(0,ktheta)] for i in range(0,krou)]
    u2irou,u2itheta = {},{}
    for u in range(0,N):
        [rou,theta] = routhetaij[u]
        irou = max(min(int((rou-minrou)/gaprou),krou-1),0)
        itheta = max(min(int((theta-mintheta)/gaptheta),ktheta-1),0)
        freqrou[irou] += 1
        u2irou[u] = irou
        if irou > 0:
            freqtheta[itheta] += 1
            u2itheta[u] = itheta
        freq[irou][itheta] += 1
    posrou = 0
    seed_start = False
    for irou in range(0,krou):
        if seed_start and freqrou[irou] > 0:
            posrou = irou
            break
        if freqrou[irou] == 0: seed_start = True
    seedrou,seedtheta = set(),set()
    for u in u2irou:
        if u2irou[u] >= posrou:
            seedrou.add(u)
    for u in u2itheta:
        seedtheta.add(u)
    return seedrou & seedtheta

In [5]:
def blame_fan_idol(nodeij,seeds,N):#edge list,seed list, node number
    D = 1.0*len(nodeij)/N/N
    n = 10
    density = (1.0/n*math.log(1.0*n/N)*2)/math.log(D)
    injectedfans,injectedidols = set(),set()
    blamedfans,blamedidols = set(seeds),set()
    lastnum_fan,lastnum_idol = 0,0
    iter_num = 0
    while not (len(blamedfans) == lastnum_fan and len(blamedidols) == lastnum_idol):
        iter_num += 1
        lastnum_fan = len(blamedfans)
        lastnum_idol = len(blamedidols)
        nodej2num = {}
        for [nodei,nodej] in nodeij:
            if nodei in blamedfans:
                if not nodej in nodej2num:
                    nodej2num[nodej] = 0
                nodej2num[nodej] += 1
        blamedidols = set()
        numedge = 0
        sort_nodej2num = sorted(nodej2num.items(),key=lambda x:-x[1])
        for item in sort_nodej2num:
            nodej,num = item[0],item[1]
            blamedidols.add(nodej)
            numedge += num
            if 1.0*numedge < density*len(blamedfans)*len(blamedidols):
                break
        nodei2num = {}
        for [nodei,nodej] in nodeij:
            if nodej in blamedidols:
                if not nodei in nodei2num:
                    nodei2num[nodei] = 0
                nodei2num[nodei] += 1
        blamedfans = set()
        numedge = 0
        sort_nodei2num = sorted(nodei2num.items(),key=lambda x:-x[1])
        for item in sort_nodei2num:
            nodei,num = item[0],item[1]
            blamedfans.add(nodei)
            numedge += num
            if 1.0*numedge < density*len(blamedfans)*len(blamedidols):
                break
        print "Iteration: ",iter_num,"; Blamed fans: ",len(blamedfans),"; Blamed idols: ", len(blamedidols)
    print "迭代了"+str(iter_num)+"轮"
#     print "precisionfan, recallfan, precisionidol, recallidol"
#     print precisionfan,recallfan,precisionidol,recallidol
    return blamedfans,blamedidols

### 3.1 选seeds

In [6]:
seeds = set()
for i in range(9,singular_k+1):
    for j in range(9,1+singular_k):
        if i==j:
            continue
        cur_seeds = select_seeds(u,i,j)
        seeds = seeds.union(cur_seeds)
    print "Iteration:",i

Iteration: 9
Iteration: 10


### 3.2 找问题节点

In [7]:
# 使用10个u得到的结果
print "seeds size:",len(seeds)
blamed_fans,blamed_idols = blame_fan_idol(nodeij,seeds,N)
print "Blamed_fans:",len(blamed_fans),"Blamed_idols:",len(blamed_idols)

seeds size: 12875


NameError: name 'nodeij' is not defined

### 3.3 转化ID到原始ID，并输出到文件中

In [92]:
bfans_origianl = [id_dict[i] for i in blamed_fans]
bidols_origianl = [id_dict[i] for i in blamed_idols]

post_fix = "u_total_10_t3"
f_name = "bm_fans_"+post_fix+".txt"
f = open(f_name,"w")
for i in bfans_origianl:
    f.write(str(i)+"\n")
f.close()

f_name = "bm_idols_"+post_fix+".txt"
f = open(f_name,"w")
for i in bidols_origianl:
    f.write(str(i)+"\n")
f.close()

In [93]:
print f_name

bm_idols_u_total_10_t3.txt


# 4. 手工查看ID对应的账号